In [61]:
# Importing libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt

In [62]:
dataset = pd.read_csv('/content/Walmart_Store_sales.csv')

In [63]:
# Basic stats
print("Nombre de ligne : {}".format(dataset.shape[0]))
print("Nombre de colonne: {}".format(dataset.shape[1]))
print()

print("Display du dataset: ")
display(dataset.head())
print()

print("Statistiques Basique: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Pourcentage de valeur manquante: ")
print()
display(100*dataset.isnull().sum()/dataset.shape[0])
print()

print('Information')
print()
display(dataset.info())

Nombre de ligne : 150
Nombre de colonne: 8

Display du dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Statistiques Basique: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Pourcentage de valeur manquante: 



Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64


Information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


None

In [64]:
### Suppression des lignes avec label manquant

dataset.dropna(subset=['Weekly_Sales'], inplace=True)

In [65]:
print("Nouveau nombre de ligne: {}".format(dataset.shape[0]))

Nouveau nombre de ligne: 136


In [66]:
print("Pourcentage de valeur manquante aprés suppression des lignes : ")
print()
display(100*dataset.isnull().sum()/dataset.shape[0])
print()

Pourcentage de valeur manquante aprés suppression des lignes : 



Store            0.000000
Date            13.235294
Weekly_Sales     0.000000
Holiday_Flag     8.088235
Temperature     11.029412
Fuel_Price       8.823529
CPI              8.088235
Unemployment    10.294118
dtype: float64

In [67]:
#To Datetime format
dataset['Date'] = pd.to_datetime(dataset['Date'], infer_datetime_format=True)

In [68]:
# Creating new columns with date values (year, month, date), to analyse each of them separately
dataset.loc[:, "Year"] = dataset["Date"].dt.year
dataset.loc[:, "Month"] = dataset["Date"].dt.month
dataset.loc[:, "Day"] = dataset["Date"].dt.day
dataset.loc[:, "Day_of_Week"] = dataset["Date"].dt.dayofweek
dataset = dataset.drop('Date', axis = 1)
dataset.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_of_Week
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [69]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 149
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         136 non-null    float64
 1   Weekly_Sales  136 non-null    float64
 2   Holiday_Flag  125 non-null    float64
 3   Temperature   121 non-null    float64
 4   Fuel_Price    124 non-null    float64
 5   CPI           125 non-null    float64
 6   Unemployment  122 non-null    float64
 7   Year          118 non-null    float64
 8   Month         118 non-null    float64
 9   Day           118 non-null    float64
 10  Day_of_Week   118 non-null    float64
dtypes: float64(11)
memory usage: 12.8 KB


In [70]:
dataset['Day_of_Week'].value_counts()

4.0    118
Name: Day_of_Week, dtype: int64

In [71]:
# Suppression de la colonne "Day_of_Week" car données non pertinentes
dataset = dataset.drop('Day_of_Week', axis=1)

In [72]:
# "Supprimons les lignes contenant des valeurs invalides ou des valeurs aberrantes. Cela concerne les colonnes : Température, Prix du carburant, CPI et Taux de chômage :"
dataset = dataset[(dataset["Temperature"] < (dataset['Temperature'].mean()+(3*dataset['Temperature'].std()))) & (dataset["Temperature"] > (dataset['Temperature'].mean()-(3*dataset['Temperature'].std())))]
display(dataset.shape[0])

dataset = dataset[(dataset["Fuel_Price"] < (dataset['Fuel_Price'].mean()+(3*dataset['Fuel_Price'].std()))) & (dataset["Fuel_Price"] > (dataset['Fuel_Price'].mean()-(3*dataset['Fuel_Price'].std())))]
display(dataset.shape[0])

dataset = dataset[(dataset["CPI"] < (dataset['CPI'].mean()+(3*dataset['CPI'].std()))) & (dataset["CPI"] > (dataset['CPI'].mean()-(3*dataset['CPI'].std())))]
display(dataset.shape[0])

dataset = dataset[(dataset["Unemployment"] < (dataset['Unemployment'].mean()+(3*dataset['Unemployment'].std()))) & (dataset["Unemployment"] > (dataset['Unemployment'].mean()-(3*dataset['Unemployment'].std())))]
display(dataset.shape[0])


121

109

102

90

In [73]:
# Displaying total revenue by year
fig = px.histogram(dataset, x='Year', y= 'Weekly_Sales')
fig.update_xaxes(type='category', categoryorder="category ascending")
fig.show()

PREPROCESSING WITH SKLEARN


In [74]:
# Separation de la variable cible
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = dataset.drop(target_variable, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

X :


,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,6.0,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0
1,13.0,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0
4,6.0,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0
6,15.0,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0
7,20.0,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0


In [75]:
# Division du dataset en Train set & Test set
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [76]:
# Creation pipeline pour les features numeric
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment','Year', 'Month', 'Day']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ])

In [77]:
# Creation pipeline pour les  features categoriel
categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [78]:
# Utilisation de ColumnTransformer pour créer un objet preprocessor qui décrit tous les traitements à effectuer.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [79]:
# Preprocessing sur l'ensemble d'entraînement
X_train = preprocessor.fit_transform(X_train)

# Preprocessing sur l'ensemble de test
X_test = preprocessor.transform(X_test)

Modele de Regression lineaire


In [80]:
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [81]:
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.93522163140001


In [82]:
regressor.coef_

array([  -11462.70403287,   -57984.82941481,   717469.9053981 ,
          32478.49680852,    -6895.02248795,    17243.19450848,
         -49592.65139628,   271338.35587122, -1250987.49342839,
        2204172.84483818, -1227432.8857967 ,   100272.53734887,
        -619830.01541625,  -656196.70312482, -1102517.25379657,
        1798457.2750563 ,   219379.59544222,  2066813.65596099,
        1017347.44167178,   589257.73223953,  -577586.04438913,
         856056.66014243,   987464.42847969,  1328210.79928435,
         592403.25635525,   -53530.32293641])

In [83]:
#get columns_names of each coefficient
column_names = []
for name, step, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = step.named_steps['encoder'].get_feature_names_out(categorical_features) # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0']


In [84]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-1.146270e+04
Fuel_Price,-5.798483e+04
CPI,7.174699e+05
Unemployment,3.247850e+04
Year,-6.895022e+03
Month,1.724319e+04
Day,-4.959265e+04
Store_2.0,2.713384e+05
Store_3.0,-1.250987e+06
Store_4.0,2.204173e+06


In [85]:


# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Year,6.895022e+03
Temperature,1.146270e+04
Month,1.724319e+04
Unemployment,3.247850e+04
Day,4.959265e+04
Holiday_Flag_1.0,5.353032e+04
Fuel_Price,5.798483e+04
Store_6.0,1.002725e+05
Store_11.0,2.193796e+05
Store_2.0,2.713384e+05


In [86]:
# Creating a pandas DataFrame to plot the feature importance from our model
dataset_coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"]).abs().sort_values(by = 'coefficients')

fig = px.bar(dataset_coefs, orientation = 'h',height=600)
fig.update_layout(showlegend = False,
                  margin = {'l': 120}) # to avoid cropping of column names
fig.show()

Regularisation Ridge


In [87]:

# Cross-validated score for a Ridge model
print("Cross-validation on Ridge...")
ridge_model = Ridge()
scores = cross_val_score(ridge_model, X_train, Y_train, cv=10)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

Cross-validation on Ridge...
The cross-validated R2-score is :  0.8475696526911364
The standard deviation is :  0.06212385843303866


In [88]:

# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] # 0 corresponds to no regularization
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.0}
Best R2 score :  0.9541101472556711


In [89]:
# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))


R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000154


Le modèle avec la régularisation Ridge offre des prédictions presque identiques à celui sans régularisation.

REGULARISATION LASSO


In [90]:
import warnings
warnings.filterwarnings('ignore')

In [91]:

# Performing grid search with Lasso regularization
regressor_lasso = Lasso()

# Grid of values to be tested
params = {'alpha': [15, 25, 35, 45, 55, 65, 75]}
gridsearch_lasso = GridSearchCV(regressor_lasso, param_grid = params, cv = 5)
gridsearch_lasso.fit(X_train, Y_train)

print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R² score : ", gridsearch_lasso.best_score_)

Best hyperparameters :  {'alpha': 25}
Best R² score :  0.9532307353371328


In [92]:

# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.9352216314000154
